# Project 02: Dual-Descent-Viz - Constrained Optimization

**Author:** Davi Bonetto  
**Course Module:** 01-Janeiro-Math  

## 1. Introduction: The Art of Constrained Optimization

In classical optimization, we often seek the global minimum of a function $f(x)$ by following the negative gradient $-\nabla f(x)$. This is akin to a ball rolling down a hill. However, real-world engineering rarely offers such unbridled freedom. Systems operate under **Constraints**: physical limits, safety bounds, or resource caps.

**Real-World Example:**  
Consider optimizing a chemical reactor. We wish to **minimize energy consumption** ($f(x)$), but we must strictly constrain the **internal temperature** to be below a critical threshold ($g(x) \leq 100^\circ C$) to prevent an explosion. A standard optimizer might minimize cost by letting the temperature rise indefinitely—a catastrophic failure.

In this project, we explore **Lagrange Multipliers** and the **Dual Ascent** method to solve such problems, visualizing the trajectory of an agent solving a "Min-Max" game.

## 2. Mathematical Formulation

We examine a non-convex optimization problem on a 2D plane $(x, y)$.

### 2.1 The Primal Objective ($f$)
We define the objective function as a hyperbolic paraboloid, often called a **Monkey Saddle** or Saddle Surface. The agent seeks to minimize this path, effectively sliding down to $-\infty$.

$$ f(x, y) = x^2 - y^2 $$

### 2.2 The Constraint ($g$)
However, we impose a hard geometric constraint. The solution must lie within the **Unit Circle**.

$$ g(x, y) = x^2 + y^2 - 1 \leq 0 $$

### 2.3 Why Gradient Descent Fails
If we naively applied Gradient Descent ($x_{t+1} = x_t - \eta \nabla f$), the agent would follow the steepest descent along the $y$-axis, quickly exiting the unit circle and diverging to negative infinity. The constraint acts as a "wall" that standard gradient descent cannot see.

## 3. Theoretical Framework: The Lagrangian Multiplier Method

To reconcile the objective $f(x)$ with the constraint $g(x)$, we construct the **Lagrangian Function** $\mathcal{L}$. This function converts the constrained problem into an unconstrained game between two variables: the primal vector $x$ and the dual variable $\lambda$ (Lagrange Multiplier).

$$ \mathcal{L}(x, \lambda) = f(x) + \lambda \cdot g(x) $$

### 3.1 The Min-Max Game (Dual Ascent)
Our goal is to find the saddle point of the Lagrangian:

$$ \min_x \max_{\lambda \geq 0} \mathcal{L}(x, \lambda) $$

This implies a competitive dynamic:
1.  **The Agent ($x$)** wants to **Minimize** $\mathcal{L}$. It descends the gradient w.r.t $x$.
    $$ x_{t+1} \leftarrow x_t - \alpha \nabla_x \mathcal{L} $$
2.  **The Environment ($\lambda$)** wants to **Maximize** $\mathcal{L}$. It ascends the gradient w.r.t $\lambda$.
    $$ \lambda_{t+1} \leftarrow \text{ReLU}(\lambda_t + \beta \nabla_\lambda \mathcal{L}) = \max(0, \lambda_t + \beta g(x)) $$

If the constraint is violated ($g(x) > 0$), $\lambda$ increases, heavily penalizing the objective. If the constraint is satisfied, $\lambda$ drops toward zero.

### 3.2 KKT Conditions (Optimality)
At the optimal solution $(x^*, \lambda^*)$, the Karush-Kuhn-Tucker (KKT) conditions must hold. Most notably **Stationarity**: the gradient of the objective must be exactly cancelled out by the gradient of the constraint times $\lambda$:
$$ \nabla f(x^*) + \lambda^* \nabla g(x^*) = 0 $$

In [ ]:
import numpy as np
import plotly.graph_objects as go

# --- 2.1 Define Functions ---

def f(x, y):
    """Objective: Saddle Surface"""
    return x**2 - y**2

def grad_f(x, y):
    """Gradient of f w.r.t [x, y]"""
    df_dx = 2 * x
    df_dy = -2 * y
    return np.array([df_dx, df_dy])

def g(x, y):
    """Constraint: Inside Unit Circle (x^2 + y^2 - 1 <= 0)"""
    return x**2 + y**2 - 1

def grad_g(x, y):
    """Gradient of g w.r.t [x, y]"""
    dg_dx = 2 * x
    dg_dy = 2 * y
    return np.array([dg_dx, dg_dy])

## 4. Implementation: The Dual Descent Engine

We implement the `DualDescentOptimizer` class. Note the distinct learning rates for the primal ($\{x, y\}$) and dual ($\{\lambda\}$) updates. Tuning these rates is critical: if $\lambda$ grows too fast, the system oscillates; if too slow, the constraint is violated for too long.

In [ ]:
class DualDescentOptimizer:
    def __init__(self, learning_rate_primal=0.05, learning_rate_dual=0.1):
        self.lr_x = learning_rate_primal
        self.lr_lambda = learning_rate_dual
        self.history = []
        
    def optimize(self, start_x, start_y, iterations=200):
        # Initialize variables
        x, y = start_x, start_y
        lam = 0.0 # Initial Lagrange Multiplier (usually 0)
        
        self.history = []
        
        for i in range(iterations):
            # 1. Compute Gradients
            gf = grad_f(x, y)
            gg = grad_g(x, y)
            constraint_val = g(x, y)
            
            # 2. Lagrangian Gradient w.r.t x (Primal step)
            # L = f + lambda * g
            # dL/dx = df/dx + lambda * dg/dx
            grad_L_x = gf + lam * gg
            
            # 3. Update primal variables (Gradient Descent)
            # x_new = x - lr * grad_L_x
            x_new = x - self.lr_x * grad_L_x[0]
            y_new = y - self.lr_x * grad_L_x[1]
            
            # 4. Update dual variable (Gradient Ascent)
            # lambda needs to increase if constraint is violated (g > 0)
            # And we project to >= 0 (KKT condition)
            lam_new = lam + self.lr_lambda * constraint_val
            lam_new = max(0.0, lam_new)
            
            # Store history
            z_val = f(x, y)
            self.history.append({
                'step': i,
                'x': x, 'y': y, 'z': z_val,
                'lambda': lam,
                'constraint': constraint_val
            })
            
            # Update state
            x, y, lam = x_new, y_new, lam_new
            
        return self.history

# --- Run Simulation ---
# Start OUTSIDE the circle at (1.5, 0.0) where objective x^2 - y^2 is 2.25
optimizer = DualDescentOptimizer(learning_rate_primal=0.02, learning_rate_dual=0.1)
history = optimizer.optimize(start_x=1.5, start_y=0.1, iterations=150)

print(f"Final Position: ({history[-1]['x']:.3f}, {history[-1]['y']:.3f})")
print(f"Final Constraint Value: {history[-1]['constraint']:.3f} (Should be <= 0)")
print(f"Final Lambda: {history[-1]['lambda']:.3f}")

## 5. Visual Analysis (Plotly)

We employ a 3D interactive visualization to observe the dynamics.
- **Surface**: The objective function terrain.
- **Cylinder/Mesh**: The boundary of the feasible region.
- **Path**: The agent's trajectory.

Pay attention to how the agent starts outside, is pulled in, and then "surfs" along the boundary wall.

In [ ]:
# Prepare Data for Plotting
traj_x = [h['x'] for h in history]
traj_y = [h['y'] for h in history]
traj_z = [h['z'] for h in history]

# 1. Surface Data (The Saddle)
x_range = np.linspace(-2, 2, 50)
y_range = np.linspace(-2, 2, 50)
X, Y = np.meshgrid(x_range, y_range)
Z = f(X, Y)

# 2. Constraint Data (The Cylinder Wall x^2 + y^2 = 1)
theta = np.linspace(0, 2*np.pi, 50)
z_cyl = np.linspace(-4, 4, 10)
THETA, Z_CYL = np.meshgrid(theta, z_cyl)
X_CYL = np.cos(THETA)
Y_CYL = np.sin(THETA)

# --- Plotly Figure ---
fig = go.Figure()

# A. The Saddle Surface
fig.add_trace(go.Surface(z=Z, x=X, y=Y, colorscale='Viridis', opacity=0.7, name='Objective'))

# B. The Constraint Wall
# We construct this using Mesh3d or just lines. Surface is easier here.
fig.add_trace(go.Surface(z=Z_CYL, x=X_CYL, y=Y_CYL, showscale=False, opacity=0.3, colorscale='Greys', name='Constraint'))

# C. The Trajectory
fig.add_trace(go.Scatter3d(
    x=traj_x, y=traj_y, z=traj_z,
    mode='lines+markers',
    marker=dict(size=4, color=list(range(len(traj_x))), colorscale='Bluered', showscale=False),
    line=dict(color='black', width=3),
    name='Agent Path'
))

# D. Start and End points
fig.add_trace(go.Scatter3d(
    x=[traj_x[0]], y=[traj_y[0]], z=[traj_z[0]],
    mode='markers', marker=dict(size=8, color='green'), name='Start'
))
fig.add_trace(go.Scatter3d(
    x=[traj_x[-1]], y=[traj_y[-1]], z=[traj_z[-1]],
    mode='markers', marker=dict(size=8, color='red'), name='End'
))

fig.update_layout(
    title="Dual Descent: Saddle Point Optimization with Constraints",
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='f(x,y)',
        aspectmode='cube'
    ),
    width=900,
    height=700
)

fig.show()

## 6. Geometric Conclusion

Observing the graph generated above, we witness the essence of **Constrained Optimization**.

1.  **Escape Attempt:** Initially, the agent (green dot) rushes down the slope, driven by $\nabla f$, violating the boundary.
2.  **Correction:** As $g(x) > 0$, the dual variable $\lambda$ spikes. This adds a gradient component $\lambda \nabla g$ pointing back towards the center.
3.  **Equilibrium:** The agent settles at the boundary (red dot). It wants to go lower (outward), but the penalty is too high. It has found the **Saddle Point** of the Lagrangian—stable in $x$, stable in $\lambda$.

This mechanic underpins modern Support Vector Machines (SVMs) and Constrained Deep Learning.